In [129]:
#импорт нужных и ненужных библиотек
import pandas as pd
from bs4 import BeautifulSoup
import webbrowser
import numpy as np
import codecs
from datetime import datetime
import calendar

In [130]:
#обработка изначального спарсенного текста: выбираем только прямые рейсы, приводим к одному формату датафрейма
df_for_32 = pd.DataFrame()
with open("C:\\Users\\dmona\\Downloads\\tuturu.txt", "r", encoding="utf-8") as file:
    for j, line in enumerate(file, start=0):
        soup = BeautifulSoup(line, "html.parser")
        data = {}
        i = 1
        spans = soup.find_all("span")
        len_spans = len(spans)
        if spans[3].text.strip() == 'прямой':
            if len_spans == 29:
                for span in spans:
                    if i != 12:
                        data[f"Column_{i}"] = [span.text.strip()]
                    else:
                        data[f"Column_{12}"] = ['']
                        data[f"Column_{13}"] = ['']
                        data[f"Column_{14}"] = ['']
                        data[f"Column_{15}"] = ['']
                        i += 3
                    i += 1
            if len_spans == 30:
                for span in spans:
                    if i != 13:
                        data[f"Column_{i}"] = [span.text.strip()]
                    else:
                        data[f"Column_{13}"] = ['']
                        data[f"Column_{14}"] = ['']
                        data[f"Column_{15}"] = ['']
                        i += 2
                    i += 1
            if len_spans == 31:
                for span in spans:
                    if span.text.strip() != '+1':
                        if i != 12:
                            data[f"Column_{i}"] = [span.text.strip()]
                        elif i == 12:
                            data[f"Column_{12}"] = ['']
                            data[f"Column_{13}"] = ['']
                            data[f"Column_{14}"] = ['']
                            data[f"Column_{15}"] = ['']
                            i += 3
                        i += 1
            if len_spans == 32:
                if spans[22].text.strip() == '+1':
                    for span in spans:
                        if span.text.strip() != '+1':
                            if i != 13:
                                data[f"Column_{i}"] = [span.text.strip()]
                            elif i == 13:
                                data[f"Column_{13}"] = ['']
                                data[f"Column_{14}"] = ['']
                                data[f"Column_{15}"] = ['']
                                i += 2
                            i += 1
                else:
                    for span in spans:
                        data[f"Column_{i}"] = [span.text.strip()]
                        i+= 1
            if len_spans == 33:
                for span in spans:
                    if i < 15:
                        data[f"Column_{i}"] = [span.text.strip()]
                    if i > 15:
                        data[f"Column_{i-1}"] = [span.text.strip()]
                    i += 1
            if len_spans == 34:
                for span in spans:
                    if i < 24:
                        data[f"Column_{i}"] = [span.text.strip()]
                    if i > 25:
                        data[f"Column_{i-2}"] = [span.text.strip()]
                    i += 1
            if len_spans == 35:
                    for span in spans:
                        if i < 10:
                            data[f"Column_{i}"] = [span.text.strip()]
                        if i > 10 and i < 25:
                            data[f"Column_{i-1}"] = [span.text.strip()]
                        if i > 26:
                            data[f"Column_{i-3}"] = [span.text.strip()]
                        i += 1
            new_row = pd.DataFrame(data)
            df_for_32 = pd.concat([df_for_32, new_row], ignore_index=True)

In [131]:
# перевод исходного датафрейма в датафрейм с понятными названиями
final_df = pd.DataFrame()

final_df['Маршрут'] = df_for_32['Column_1']
final_df['Время в пути'] = df_for_32['Column_6']
final_df['Авиакомпания'] = df_for_32['Column_9']
final_df['Рейс'] = df_for_32['Column_11']
final_df['Модель самолета'] = df_for_32['Column_13']
final_df['Время вылета'] = df_for_32['Column_18']
final_df['Дата вылета'] = df_for_32['Column_19']
final_df['Откуда'] = df_for_32['Column_20']
final_df['Время прилета'] = df_for_32['Column_22']
final_df['Дата прилета'] = df_for_32['Column_23']
final_df['Куда'] = df_for_32['Column_24']
final_df['Цена билета'] = df_for_32['Column_27']
final_df['Ручная кладь'] = df_for_32['Column_29']
final_df['Багаж'] = df_for_32['Column_30']
final_df['Возврат'] = df_for_32['Column_31']
final_df['Обмен'] = df_for_32['Column_32']

In [132]:
#перевод всех элементов в датафрейме в удобный формат строк
def normal_string(kringe):
    kringe = str(kringe)
    kringe = kringe.replace('\u202f', '')
    kringe = kringe.replace('\xa0', '')
    return kringe
final_df = final_df.applymap(normal_string)
final_df['Время вылета'] = final_df['Время вылета'].apply(lambda x: None if x == '' else x)
final_df = final_df.dropna()

C:\Users\dmona\AppData\Local\Temp\ipykernel_12320\3422417952.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  final_df = final_df.applymap(normal_string)


In [133]:
#функция для обработки времени в пути
def time_in_air (stringe):
    if stringe.find('м') != -1:
        return int(stringe[0: stringe.find('ч')]) * 60 +  int(stringe[stringe.find('ч') + 1: stringe.find('м')])
    else:
        return int(stringe[0: stringe.find('ч')]) * 60
final_df['Время в пути'] = final_df['Время в пути'].apply(time_in_air)

In [134]:
#функция для обработки дат вылета и прилета
days_of_week = {'пн': 1, 'вт': 2, 'ср': 3, 'чт': 4, 'пт': 5, 'сб': 6, 'вс': 7}
months = {'июня': '06 ', 'июля' : '07 ', 'августа': '08 '}
def normal_date(cringe_date):
    if cringe_date != '':
        if cringe_date[0].isdigit():
            day = cringe_date[0: cringe_date.find(' ')]
            month = cringe_date[cringe_date.find(' ') + 1: cringe_date.find(',')]
            day_of_week = cringe_date[cringe_date.find(',') + 2:]
            month = months[month]
            date = month + day
            day_of_week = days_of_week[day_of_week]
            return datetime.strptime('2024 '+ month + day, '%Y %m %d'), day_of_week
    else:
        return None, None

final_df['Дата вылета'], final_df['День недели вылета'] = zip(*final_df['Дата вылета'].apply(normal_date))

final_df['Дата прилета'], final_df['День недели прилета'] = zip(*final_df['Дата прилета'].apply(normal_date))

In [135]:
#перевод цены билета в численный формат
def normal_price(cringe):
    cringe = cringe.replace('\u2009\u2060', '')
    cringe = cringe.replace('₽', '')
    return float(cringe)
final_df['Цена билета'] = final_df['Цена билета'].apply(normal_price)

In [136]:
#работа с размером багажа
luggage ={'Без багажа': 0, 'Багаж 23кг' : 23,  'Багаж 32кг': 32, 'Багаж 2×23кг' : 46, 'Багаж 10кг' : 10, 
 'Багаж 20кг' : 20, 'Багаж 30кг' : 30, 'Багаж 15кг': 15}
final_df['Багаж'] = final_df['Багаж'].apply(lambda x: luggage[x])


In [137]:
#работа с возвратом билетов
return_tickets = {'Без возврата' : 0, 'Возврат сосбором': 1}
final_df['Возврат'] = final_df['Возврат'].apply(lambda x: return_tickets[x])

In [138]:
#работа с ручной кладью
hand_luggage = {'Ручная кладь 5кг': 5, 'Ручная кладь 10кг': 10}
final_df['Ручная кладь'] = final_df['Ручная кладь'].apply(lambda x: hand_luggage[x])

In [139]:
#работа с обменом билетов
exchange = {'Без обмена': 0,'Обмен сосбором': 1}
final_df['Обмен'] = final_df['Обмен'].apply(lambda x: exchange[x])

In [141]:
#перевод времени в формат ЧЧ:ММ
final_df['Время вылета'] = final_df['Время вылета'].apply(lambda x: datetime.strptime(x, '%H:%M').time())
final_df['Время прилета'] = final_df['Время прилета'].apply(lambda x: datetime.strptime(x, '%H:%M').time())

In [ ]:
#открытие датафрейма в браузере
html_table = final_df.to_html()
with open("df_table.html", "w", encoding="utf-8") as file:
    file.write(html_table)
webbrowser.open("df_table.html")